In [72]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [73]:
ratings = pd.read_csv('09_AE_03_RatingData.csv')
ratings = ratings[['userId', 'movieId', 'rating']]

ratings_matrix = ratings.pivot(# 각 영화를 피쳐로, 점수를 값으로
    index='userId',
    columns='movieId',
    values='rating')

# 평가 되지 않은 항목은 0으로 weight 과 곱해질 때 이 피쳐와 weight 의 곱이 0으로 되기 때문
ratings_matrix.fillna(0, inplace=True)

# 정규화 ratings_matrix 는 약 10000개의 피쳐가 있다 (영화의 갯수 및 평)
ratings_matrix = ratings_matrix.values / 5.0

In [81]:
# 오토인코더 모델 생성
input_layer = Input(shape=(ratings_matrix.shape[1],))
encoded = Dense(512, activation='relu')(input_layer)

encoded = Dense(256, activation='relu')(encoded) # 잠재 공간

decoded = Dense(512, activation='relu')(encoded) # 디코더
output_layer = Dense(ratings_matrix.shape[1],
                     activation='sigmoid')(decoded) # 평점이 0과 1 사이의 값이 되도록

# 원본 평점과 예측 평점 간의 차이를 최소화하도록 학습 유도
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [82]:
# 훈련 및 검증 데이터 분리
x_train, x_val = train_test_split(ratings_matrix, test_size=0.2, random_state=42)

In [83]:
# 모델 학습
# 입력 평점을 재구성하는 방법을 학습
autoencoder.fit(x_train, x_train,epochs=50,batch_size=256,validation_data=(x_val, x_val))


Epoch 1/50
2/2 [==============================] - 1s 160ms/step - loss: 0.2471 - val_loss: 0.2430
Epoch 2/50
2/2 [==============================] - 0s 46ms/step - loss: 0.2429 - val_loss: 0.2246
Epoch 3/50
2/2 [==============================] - 0s 47ms/step - loss: 0.2215 - val_loss: 0.1660
Epoch 4/50
2/2 [==============================] - 0s 53ms/step - loss: 0.1641 - val_loss: 0.0913
Epoch 5/50
2/2 [==============================] - 0s 50ms/step - loss: 0.0849 - val_loss: 0.0372
Epoch 6/50
2/2 [==============================] - 0s 49ms/step - loss: 0.0300 - val_loss: 0.0162
Epoch 7/50
2/2 [==============================] - 0s 48ms/step - loss: 0.0113 - val_loss: 0.0126
Epoch 8/50
2/2 [==============================] - 0s 55ms/step - loss: 0.0084 - val_loss: 0.0121
Epoch 9/50
2/2 [==============================] - 0s 47ms/step - loss: 0.0083 - val_loss: 0.0119
Epoch 10/50
2/2 [==============================] - 0s 43ms/step - loss: 0.0083 - val_loss: 0.0119
Epoch 11/50
2/2 [===========

In [84]:
# 모델 평가
val_loss = autoencoder.evaluate(x_val, x_val)
print("Validation loss:", val_loss)

4/4 [==============================] - 0s 3ms/step - loss: 0.0120
Validation loss: 0.0120133887976408


In [85]:
# 평점 예측
# 학습된 모델을 사용하여 사용자 평점 예측
predicted_ratings = autoencoder.predict(ratings_matrix) * 5.0  # 스케일을 원래대로 복구

20/20 [==============================] - 0s 1ms/step


In [88]:
# 사용자가 아직 평가하지 않은 영화에 대한 추천 생성
def generate_recommendations(user_id, predicted_ratings, ratings_matrix):
    user_ratings = ratings_matrix[user_id - 1]  # 사용자의 평점 데이터 가져오기 (인덱스는 0부터 시작하므로 -1)
    unseen_movies_idx = user_ratings == 0  # 사용자가 아직 평가하지 않은 영화 찾기
    predicted_ratings_for_user = predicted_ratings[user_id - 1]  # 사용자에 대한 예측 평점 가져오기
    recommended_movies = predicted_ratings_for_user[unseen_movies_idx]  # 아직 보지 않은 영화에 대한 예측 평점 가져오기
    top_recommended_movie_ids = recommended_movies.argsort()[::-1][:10]  # 상위 10개 추천 영화 선택
                                      # argsort()[::-1]는 평점을 내림차순으로 정렬,
                                      # 가장 높은 평점을 가진 10개의 영화 인덱스 반환

    # print(unseen_movies_idx) # 평점 0 True, 0이 아닌(평점이 있는 영화) index는 False
    return top_recommended_movie_ids

# 예시: 사용자 1에 대한 영화 추천 생성
user_id = 1
recommendations = generate_recommendations(user_id, predicted_ratings, ratings_matrix)
        # user_id: 사용자 ID
        # predicted_ratings: 모델을 통해 생성된 예측 평점
        # ratings_matrix: 사용자-영화 평점 matrix(원 데이터) -평가하지 않은 영화(0값) 정보 포함
print("Recommended movie IDs for user {}: {}".format(user_id, recommendations))

Recommended movie IDs for user 1: [ 259  615 3402 3162 3168 3167 3166 3165 3164 3163]
